## File containing funcitons for extracting data from dataset

In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
import pandas as pd
import xlrd

In [2]:
def extarctChannelNames(channelFile):
    channelNames=channelFile.iloc[:,0]
    channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
    channelNames.insert(0,"FP1")
    """
    Function that extracts all the channel names specfied from path (Assumes that the channel names are in the first column)
        
    Parameters
    ----------
    channelFile : str
        The directory containing the channel names
        
    Returns
    -------
    channelNames : list
        The list containg all channel names used in the 10-20 standard
    """
    return channelNames

In [3]:
def getLabelsAndMatfiles(matfiles):
    matfiles.sort()
    labels=loadmat(matfiles[15])
    matfiles.pop()
    label_list=np.ndarray.tolist(labels['label'])[0]
    """
    Function that sorts all the file names and extracts the label file
        
    Parameters
    ----------
    matfiles : list
        The list containing all names for data files in the dataset. 
        
    Returns
    -------
    matfiles : list
        The sorted list of data files excluding the label files.
    label_list : list
        The labels represented as a list. 
    """
    return matfiles, label_list

In [4]:
def getRawArrayData():
    channelNames=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    matfiles,label_list = getLabelsAndMatfiles(glob.glob('Preprocessed_EEG/*.mat'))
    info=mne.create_info(channelNames,200,'eeg')
    subjects={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    for x in range(0,15):
        clip_info={}
        used_data = loadmat(matfiles[x])
        del used_data["__header__"]
        del used_data["__version__"]
        del used_data["__globals__"]
        keyName=list(used_data.keys())[0][:-1]
        for ind,i in enumerate(indexs):
            rawData=used_data[keyName + str(i)]
            clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)
        subjects[x]=clip_info
    """
    Function that extracts all the data from the given dataset
        
    Returns
    -------
    subjects : dict
        All data from the dataset in the form of rawArrays stored within dictionary entries
    """
    return subjects